In [304]:
# DO NOT modify this cell. 
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

# Part I: Vertical seam removal only
---

In [297]:
# Parses the .pgm file returning only the array of pixels
def getImageArray(filename2):

    with open(filename2, 'r') as file:
        # throw away first two lines
        file.readline()
        file.readline()

        # get the size of the image
        arraySize = file.readline().split()

        width = int(arraySize[0])
        height = int(arraySize[1])

        # create an empty array of size row * column
        image = [[0 for _ in range(width)] for _ in range(height)]

        # throw away the fourth line
        file.readline()

        # read file into a temp array to be iterated over
        tempArray = file.read().split()

        counter = 0

        for row in range(height):
            for column in range(width):
                image[row][column] = int(tempArray[counter])
                counter += 1

        return image

# Calculates and returns the energy map
def getEnergyMap(image):

    energyMap = [[0 for _ in range(int(len(image[0])))] for _ in range(int(len(image)))]

    height = int(len(image))
    width = int(len(image[0]))
    
    for y in range(height):
        for x in range(width):

            # Calculate energy in x direction

            # left edge
            if (x == 0):
                dx = abs(image[y][x+1] - image[y][x])
            elif (x == width - 1):
                # right edge
                dx = abs(image[y][x] - image[y][x-1])
            else:
                # everything else
                dx = abs(image[y][x] - image[y][x-1]) + abs(image[y][x] - image[y][x+1])
        
            # Calculate energy in y direction

            # top edge
            if (y == 0):
                dy = abs(image[y][x] - image[y + 1][x])
            # bottom edge
            elif (y == height - 1):
                dy = abs(image[y][x] - image[y - 1][x])
            # everything else
            else:
                dy = abs(image[y][x] - image[y - 1][x]) + abs(image[y][x] - image[y + 1][x])

            energyMap[y][x] = abs(dy) + abs(dx)

    return energyMap

# Calculates and returns a top-to-bottom minimum seam
def findMinimumSeam(energyMap):
    height, width = len(energyMap), len(energyMap[0])

    # empty 2D array to store the sums
    total = [[0 for _ in range(width)] for _ in range(height)]

    # copy the first row into the total since we can't consider anything above them
    total[0] = energyMap[0]

    # iterate over the remaining rows of the energy map to calculate the cumulative energy values
    for y in range(1, height):
        for x in range(width):
            # Calculate the cumulative energy value for the current pixel by adding the current energy value and the minimum of the three possible values from the row above

            # left edge
            if x == 0:
                total[y][x] = energyMap[y][x] + min(total[y-1][x], total[y-1][x+1])
            # right edge
            elif x == width - 1:
                total[y][x] = energyMap[y][x] + min(total[y-1][x], total[y-1][x-1])
            # everything else
            else:
                total[y][x] = energyMap[y][x] + min(total[y-1][x-1], total[y-1][x], total[y-1][x+1])

    # Find the minimum energy value in the last row of the total array
    minimumEnergy = min(total[height-1])

    # Find the index of the minimum energy value in the last row. If there are two minimums, choose the first one encountered
    minimumSeam = []
    for x in range(width):
        if total[height-1][x] == minimumEnergy:
            minimumSeam.append(x)
            break
    
    # Backtrack to find the vertical minimumSeam
    # second last row (height-2) to the first row (-1) in reverse order (-1)
    for y in range(height - 2, -1, -1):
        x = minimumSeam[-1]
        
        # left edge
        if x == 0:
            if total[y][x] <= total[y][x+1]:
                minimumSeam.append(x)
            else:
                minimumSeam.append(x+1)
        # right edge
        elif x == width - 1:
            if total[y][x-1] <= total[y][x]:
                minimumSeam.append(x-1)
            else:
                minimumSeam.append(x)
        # everything else
        else:
            min_neighbor = min(total[y][x-1], total[y][x], total[y][x+1])

            if min_neighbor == total[y][x-1]:
                minimumSeam.append(x-1)
            elif min_neighbor == total[y][x]:
                minimumSeam.append(x)
            elif (min_neighbor == total[y][x+1]):
                minimumSeam.append(x+1)
    
    # Reverse the minimumSeam to get the correct order from top to bottom
    minimumSeam = minimumSeam[::-1]
    return minimumSeam

# Removes the seam of pixels
def removeSeam(minimumSeam, img):

    for y in range(len(img)):
        img[y].pop(minimumSeam[y])

    return img


image = getImageArray(filename)

for i in range(verticalSeam2Remove):
    energyMap = getEnergyMap(image)
    minimumSeam = findMinimumSeam(energyMap)
    image = removeSeam(minimumSeam, image)

processedFilename = filename.split(".")[0]+"_processed_"+str(verticalSeam2Remove)+"_0.pgm"

# Write to output file
with open(processedFilename, 'w') as file:
    file.write('P2\n')
    file.write(f'{len(image[0])} {len(image)}\n')
    file.write('255\n')
    for row in image:
        for column in row:
            file.write(str(column))
            file.write(" ")
        file.write("\n")

# Part II : both vertical and horizontal seams removal 
---

In [306]:
filename = "CAS_2.pgm"
image = getImageArray(filename)

for i in range(verticalSeam2Remove):
    energyMap = getEnergyMap(image)
    minimumSeam = findMinimumSeam(energyMap)
    image = removeSeam(minimumSeam, image)

# Transpose the image
image = [[row[i] for row in image] for i in range(len(image[0]))]

for i in range(horizontalSeam2Remove):
    energyMap = getEnergyMap(image)
    minimumSeam = findMinimumSeam(energyMap)
    image = removeSeam(minimumSeam, image)

# Transpose the image back
image = [[row[i] for row in image] for i in range(len(image[0]))]

processedFilename = filename.split(".")[0]+"_processed_daniel_"+str(verticalSeam2Remove)+"_"+str(horizontalSeam2Remove)+".pgm"

# Write to output file
with open(processedFilename, 'w') as file:
    file.write('P2\n')
    file.write(f'{len(image[0])} {len(image)}\n')
    file.write('255\n')
    for row in image:
        for column in row:
            file.write(str(column))
            file.write(" ")
        file.write("\n")